## Various Number Theory Experiments

_burt rosenberg
<br>
1 november 2019_


### Congruence classes

In [34]:
import random

#
# for integers mod n, a congruence class is the set of integers
# of the same remainder when divided by n, 
#    { i+j*n | j is any integer }
# the arithematic of congruence classes is defined from integer
# arithmetic by taking any integer, one from each of the two
# congruence classes, performing the integer calculation (add or
# multiply), then taking as result the congruence class that 
# contains that integer result
#
# it is an easy enough proof. however the following is an experiment
# to help conceptualize the proof.
#

def congruence_add_p(A,B,C):
    """
    does A + B = C, where A, B and C are congruence classes
    """
    s = random.choice(A[:5])+random.choice(B[:5])
    return s in C

def build_congruence(n):
    c_g = [i for i in range(n)]
    for i in range(n):
        c_g[i] = [i+j for j in range(0,15*n,n)]
    return c_g

def test_congruence_add_p(n):
    c_g = build_congruence(n)
    for i in range(n):
        for j in range(n):
            for k in range(3):
                if not congruence_add_p(c_g[i],c_g[j],c_g[(i+j)%n]):
                    print("***fail***")
        for i in range(n):
            for j in range(n):
                for k in range(3):
                    if congruence_add_p(c_g[i],c_g[j],c_g[(i+j+k+1)%n]):
                        print("***fail***")

    print("***pass***")
    
test_congruence_add_p(5)

***pass***


### Bezouts relationship

In [39]:
#
# the set { i*a + j*b | for i and j all integers} is the
# same as the set { gcd(a,b)*i | for i all integers }
# 
# this is useful in finding a gcd, as once one can start at the
# two elements of this set, a and b, and "climb down" the ladder
# of integers in the set until one arrives at the rung just above
# zero. that is the gcd.
#

def integer_span(a,b,n,width=10):
    C = []
    for i in range(-n,n):
        for j in range(-n,n):
            d = i*a+j*b
            if d not in C:
                C.append(d)
    C = sorted(C)
    return [i for i in filter(lambda x: x<width and x>-width, C)]


print(integer_span(7,9,5))
    
    

[-9, -8, -7, -6, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


### Extended gcd algorithm

In [41]:
#
# using the bezouts set, not only can we find the gcd
# but we can find the promised s and t where,
#   gcd(a,b) = s*a + b*t
# as we work down the bezouts set towards zero, but jumping
# from remainder to remainder, we keep track of the quotients
# involved in the remainder calculation.
#


def extended_gcd(a,b):
    """
    extended GCD algorithm. recursive.
    returns (d,s,t) where d = s*a+t*b 
    and d = gcd(a,b)
    """
    assert(
        a>=0 and b>=0 )
    if b==0:
        return (a,1,0)
    (q,r) = divmod(a,b)
    (d,s,t) = extended_gcd(b,r)
    # gcd(a, b) == gcd(b, r) == s*b + t*r == s*b + t*(a - q*b)
    return (d,t,s-q*t)


def test_e_gcd(n):
    for i in range(n):
        (d,s,t) = extended_gcd(i,n)
        if d==1:
            # check the inverse property
            if (i*s%n)!=1:
                print("***failed***")
                return
        else:
            # check the divisibility property
            if i%d!=0 or n%d!=0:
                print("***failed***")
                return
    print("***passed***")

            
test_e_gcd(100000)


***passed***


### Subgroups generated by an element

In [43]:
#
# using the orbit of and element g in mod n, <g>, we first
# note how they are subgroups, we also not Legrange property that
# the orbit, as a subgroup, divides the size of the containing group,
# and we see that it is the kernel of a set of co-sets that neatly
# organizes the group into congruence classes, as happened with 
# the integers mod n, them selves.
#
# in preparation for a discussion of little fermat, we generate
# the orbit not by chasing an element around, g, g^2, g^3 etc
# but by taking the enter map of Z_n -> Z-n, multiplication by
# g, and noting that it is a permutation.
#
# which we then organize into cycle notation. the cycle with 1
# is <g>, and the other cycles are the cosets
#
# we can also make remarks about the action of this self-map 
# "multiplication by g" on the non-invertibles of Z_n, which
# don't otherwise form a subgroup, for lack of an invertible
# operation
#
#

def invertibles(n):
    xr = [i for i in 
          filter(lambda x: (extended_gcd(x,n)[0]==1),
                 range(1,n))]
    xnr = [i for i in filter(lambda x: (x not in xr),
                 range(n))]
    return xr, xnr


def generalized_orbit(g,n):
    """
    The generalized orbit of g mod n is the permutation on Zn 
    multiplication by g, x goes to x*g%n.
    """
    o = [1]
    if extended_gcd(g,n)[0]!=1:
        return o
    if g!=1:
        o += [g]
        while (o[-1]*g)%n!=1:
            o += [(o[-1]*g)%n]
    O = [o[:]]
    
    def flatten(O):
        l = []
        for o in O:
            l += o
        return l

    xr, xnr = invertibles(n)
    for l in range(0,len(xr)//len(o)):
        for x in xr:
            if x not in flatten(O):
                O += [[j*x%n for j in o]]
    return O

def visualize_orbit(n):
    xr, xrn = invertibles(n)
    #print("inv:",xr)
    print([xrn[0]],"\n   non-invertibles:",xrn[1:])
    g_o = []
    for g in xr:
        g_o += [generalized_orbit(g,n)]
    g_o = [y for (x,y) in sorted([(len(x),x) for x in g_o],reverse=True)]
    for g in g_o:
        if len(g)==1:
            print(g[0],"\n   ",g[0][1],"generates group")
            
        elif len(g[0])==1:
            print(g[0],"\n   invertibles:",g[1:])
        else:
            print(g[0],"\n   cosets:")
            for g1 in g[1:]:
                print("  ",g1)

def noninvt(n):
    xn, xnr = invertibles(n)
    print("\ninvertible times an non-invertible")
    for x in xn:
        print(x,[i*x%n for i in xnr])
    print("\nproduct of non-invertibles")
    for x in xnr:
        if x==0:
            continue
        print(x,[i*x%n for i in xnr])
        
visualize_orbit(15)
noninvt(15)

[0] 
   non-invertibles: [3, 5, 6, 9, 10, 12]
[1] 
   invertibles: [[2], [4], [7], [8], [11], [13], [14]]
[1, 14] 
   cosets:
   [2, 13]
   [4, 11]
   [7, 8]
[1, 11] 
   cosets:
   [2, 7]
   [4, 14]
   [8, 13]
[1, 4] 
   cosets:
   [2, 8]
   [7, 13]
   [11, 14]
[1, 13, 4, 7] 
   cosets:
   [2, 11, 8, 14]
[1, 8, 4, 2] 
   cosets:
   [7, 11, 13, 14]
[1, 7, 4, 13] 
   cosets:
   [2, 14, 8, 11]
[1, 2, 4, 8] 
   cosets:
   [7, 14, 13, 11]

invertible times an non-invertible
1 [0, 3, 5, 6, 9, 10, 12]
2 [0, 6, 10, 12, 3, 5, 9]
4 [0, 12, 5, 9, 6, 10, 3]
7 [0, 6, 5, 12, 3, 10, 9]
8 [0, 9, 10, 3, 12, 5, 6]
11 [0, 3, 10, 6, 9, 5, 12]
13 [0, 9, 5, 3, 12, 10, 6]
14 [0, 12, 10, 9, 6, 5, 3]

product of non-invertibles
3 [0, 9, 0, 3, 12, 0, 6]
5 [0, 0, 10, 0, 0, 5, 0]
6 [0, 3, 0, 6, 9, 0, 12]
9 [0, 12, 0, 9, 6, 0, 3]
10 [0, 0, 5, 0, 0, 10, 0]
12 [0, 6, 0, 12, 3, 0, 9]


### Euler Phi Function and Little Fermat

In [45]:
#
# the euler phi function of n counts the number of elements
# relativey prime to n in the range 1 to n-1. there is a formula
# to efficiently calculate phi, but here we content ourselves 
# to make a list of all invertibles in Z_n and phi is the number
# of elements on the list.
#
# because we know Z_n goes to Z_n as a permutation by the action
# multiplication by an invertible, we get Euler's Theorem, which
# is a generalization of Little Fermat to composite moduli.
#
# also noted is wilson's theorem, also a consequence of this
# self-action, which concerns the factorial (n-1)! in Z_n.
#

def euler_phi_function(n):
    """
    phi(n) = n Prod (1-1/p), all primes p|n.
    """
    return len(invertibles(n)[0])

def proof_of_eulers_theorem(n):
    """
    Euler's is a generalization of little fermat for 
    any n. its proof can be that the map Zn->Zn multiplication
    by a where a is rel prime to n, is a permutation.
    
    Little fermat is the case n is a prime.
    """
    xn, xnr = invertibles(n)
    # phi = len(xn)
    phi = euler_phi_function(n)
    for x in xn:
        p = [x*i%n for i in xn]
        if sorted(p)!=sorted(xn):
            print("***fail***")
        if pow(x,phi,n)!=1:
            print("***fail***")
    print("***passed fermat test***")
    
def wilsons_theorem(n):
    """
    Gauss proved the generalization that 
    the product of all numbers relatively
    prime to n between 1 and n-1, 
    is -1 in the cases 
    - the power of an odd prime
    - twice such a number
    - or 4
    and 1 in all other cases.
    """
    xn, xnr = invertibles(n)
    p = 1
    for x in xn:
        p = (p*x)%n
    if p==(n-1):
        p = -1
    if p==1:
        print("***not a prime***")
    elif p==-1:
        print("***a prime power, twice a prime power, or 4***")
    else:
        print("***fail***")          
        

proof_of_eulers_theorem(113)
wilsons_theorem(113)


***passed fermat test***
***a prime power, twice a prime power, or 4***


### Primality Testing

In [50]:
#
# in cases we need a prime number, we must have an efficient
# way of determining if a number is prime. trial divisor or
# the Sieve of Eratosthenes is not efficient.
#
# using the converse of Little Fermat is an idea, but fails because
# of Carmichael numbrers. Miller-Rabin is similar, but does not fail.
#
# these are PPT algorithms, that express their randomness in the
# possibility of a one-sided error. The algorithms randomly draw
# from Z_n to see if it is a witness to compositeness. There are
# no witnesses to primality, except that repeated trials has
# failed to find a witness to compositeness.
#
# the key to miller rabin is a conductor to 1, by doing the
# exponentiation of little fermat several steps. it will lead
# necessarily to one in a certain way for a prime, but not
# necessarilty in that one for a composite
#
def miller_rabin_conductor(x,n):
    if (n%2==0):
        return []
    assert(x>0)
    s = n-1
    t = 0
    while s%2==0:
        s = s//2
        t += 1
    assert(s%2!=0)
    l = [x]
    x = pow(x,s,n)
    l.append(x)
    if x==1 or x==-1:
        return l
    for i in range(t):   
        x = pow(x,2,n)
        l.append(x)
    if x!=1:
        x = pow(x,2,n)
        l.append(x)        
    return l

print(miller_rabin_conductor(2,15))
        
        
        

[2, 8, 4, 1]
